In [ ]:
import pandas as pd
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scikit_posthocs as sp
from statannot import add_stat_annotation
from itertools import combinations
from scipy import stats

### Data import

In [ ]:
#Import of CDS sequences from Araport 
from collections import defaultdict

dic=defaultdict(str)
with open("C:/.../Araport11_cds_20220914") as f:
    line='1'
    while line:
        line=f.readline().strip()
        if ">" in line:
            name=line[1:line.index("|")-1]
            continue
        dic[name]+=line      

cds=dict(dic)
cds=dict(filter(lambda x: len(x[1])%3==0 and set(x[1])-{"A",'C',"G","T"}==set() and ".1" in x[0] and x[1][:3]=="ATG" and len(x[1])>3 and x[1][-3:]in ['TGA','TAA','TAG'], cds.items()))

codons=dict(map(lambda x: [x[0],[x[1][i:i+3] for i in range(0,len(x[1]),3)]],cds.items()))

all_codons=[j for i in codons.values() for j in i]

codon_freq=Counter(all_codons)

In [ ]:
#Import of the T-RIP dataset
df=pd.read_excel('C:/.../T-RIP_data', skiprows=[0])

### Setting up dependencies

In [ ]:
Gen_code={
    'Phe': ["TTT","TTC"],
    'Leu':["TTA","TTG","CTT","CTC","CTA",'CTG'],
    'Ile':["ATT","ATC","ATA"],
    'Met':["ATG"],
    'Val':["GTT","GTC","GTA","GTG"],
    'Ser':["TCT","TCC","TCA",'TCG',"AGT",'AGC'],
    'Pro':["CCT","CCC","CCA","CCG"],
    'Thr':['ACT',"ACC","ACA","ACG"],
    'Ala':["GCT","GCC","GCA","GCG"],
    'Tyr':["TAT","TAC"],
    'Ter':["TAA","TAG","TGA"],
    'His':["CAT","CAC"],
    'Gln':["CAA","CAG"],
    'Asn':["AAT","AAC"],
    'Lys':["AAA","AAG"],
    'Asp':["GAT","GAC"],
    'Glu':["GAA","GAG"],
    'Cys':["TGT",'TGC'],
    'Trp':["TGG"],
    'Arg':["CGT","CGC","CGA","CGG","AGA","AGG"],
    'Gly':['GGT',"GGC","GGA","GGG"]
}


In [ ]:
l=[(i,j) for i in range(3) for j in ["A","G","C","T"]]
nucleotide_freqs={nucleotide+"_"+str(position):sum(map(lambda y: y[1], filter(lambda x: x[0][position]==nucleotide, codon_freq.items())))/len(all_codons) for position,nucleotide in l}

### Calculation of RCBS score

In [ ]:
#To run properly, this class is dependent on aforementioned
import numpy as np
from collections import Counter

class RCBS:
    def __init__(self, seq):
        self.seq=seq
        
    def __len__(self):
        return len(self.seq)
    
    def codon_frequency(self):
        return dict(Counter(self.seq))
    
    def relative_codon_frequency(self):
        return {k:v/len(self.seq) for k,v in Counter(self.seq).items()}

    def nucleotide_frequencies(self):
        l=[(i,j) for i in range(3) for j in ["A","G","C","T"]]
        return {nucleotide+"_"+str(position):sum(map(lambda y: y[1], filter(lambda x: x[0][position]==nucleotide,Counter(self.seq).items())))/len(self.seq) for position,nucleotide in l}
    
    def translation_table(self):
        return {item:k for k,v in Gen_code.items() for item in v}
    
    def gene_code(self):
        return dict(map(lambda x: (x[0], [item for item in x[1] if item in self.seq]), Gen_code.items()))
    
    def codon_RCBS(self):
        return dict(map(lambda x: (x[0], {i:self.relative_codon_frequency()[i]/np.prod([self.nucleotide_frequencies()[f"{i[j]}_{j}"] for j in range(3)]) for i in x[1]}), self.gene_code().items()))
    
    def corrected_codon_RCBS(self):
        return dict(map(lambda x: (x[0], {i:((self.relative_codon_frequency()[i]*len(self.seq)+rela[i]*500)/(len(self.seq)+500))/np.prod([(self.nucleotide_frequencies()[f"{i[j]}_{j}"]*len(self.seq)+nucleotide_freqs[f"{i[j]}_{j}"]*500)/(len(self.seq)+500) for j in range(3)]) for i in x[1]}), self.gene_code().items()))
        
    
    def gene_corrected_RCBS(self):
        RCBS_simplified={k:v for x in self.corrected_codon_RCBS().values() for k,v in x.items()}
        values={k:v**p for k,v in RCBS_simplified.items() for q,p in self.codon_frequency().items() if k==q}
        return np.prod(list(values.values()))**(1/len(self.seq))-1

In [ ]:
#Calculation of length normalized RCBS score
RCBS_vals={k: RCBS(v).gene_corrected_RCBS() for k,v in codons.items()}

### LogFC binning based on RCBS

In [ ]:
#For heat conditions "Log ratio heat" may be used
mock_enriched=df[df["Log ratio mock"]>1]["geneID"].to_list()

In [ ]:
to_1_5=df[(df["Log ratio mock"]>1) & (df["Log ratio mock"]<=1.5)]["geneID"].to_list()
to_2=df[(df["Log ratio mock"]>1.5) & (df["Log ratio mock"]<=2)]["geneID"].to_list()
up_2=df[(df["Log ratio mock"]>2)]["geneID"].to_list()

In [ ]:
first=list(map(lambda x: x[1],filter(lambda x: x[0][:-2] in to_1_5, RCBS_vals.items())))
second=list(map(lambda x: x[1],filter(lambda x: x[0][:-2] in to_2, RCBS_vals.items())))
third=list(map(lambda x: x[1],filter(lambda x: x[0][:-2] in up_2, RCBS_vals.items())))

In [ ]:
results=np.triu(sp.posthoc_dunn([first,second,third], p_adjust = 'bonferroni'),1)
indices=np.nonzero(results)
p_vals=results[indices].tolist()

In [ ]:
ps=list(map(lambda x: "ns"if x>=0.05 else "$"+f"{x:.2e}"f"{x}"[:f"{x:.2e}".index("-")].replace("e","\cdot10^{%s}"%f"{x:.2e}"[f"{x:.2e}".index("-"):])+"$",p_vals))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

tags=['1-1.5',"1.5-2",">2"]
sns.set_style("white")

sns.histplot(data=[first,second,third],bins=50,kde=True,palette=["#87CEEB","#FFA500","#0CBAA6"],alpha=0.8)
#sns.kdeplot(data=[first,second,third],palette=["#87CEEB","#FFA500","#0CBAA6"],fill=True)

ax.axvline(np.median(first),linestyle='--',color="#87CEEB", alpha=0.8)
ax.axvline(np.median(second),linestyle='--',color="#FFA500", alpha=0.8) #For fixed grouping
ax.axvline(np.median(third),linestyle='--',color='#0CBAA6', alpha=0.8)

ax.legend(labels=tags[::-1],ncols=3) 
ax.set_title('Mock')
ax.set_xlabel("RCBS")

ax_sub=fig.add_axes([0.57,0.4,0.32,0.35])
sns.set_style("whitegrid")

sec=sns.boxplot(data=[first,second,third], 
            showfliers=False,palette=["#87CEEB","#FFA500","#0CBAA6"])
#sns.stripplot(data=[first,second,third],color="k", 
            #size=2,alpha=0.8)

add_stat_annotation(ax=sec,x=tags,y=[np.median(first),np.median(second),np.median(third)],
                                           box_pairs=list(combinations(tags,2)),
                                           pvalues=p_vals,
                                           text_annot_custom=ps,
                                           text_format="simple",
                                           loc="inside",
                                           perform_stat_test=False,
                                           line_offset_to_box=0.65)



ax_sub.set_xticklabels(tags)

plt.xticks(rotation=45)

#plt.savefig("name.tiff",dpi=600)

### Pairwise comparison between enriched transcripts and transcriptome

In [ ]:
enriched=list(map(lambda x:x[1],filter(lambda x: x[0][:-2] in mock_enriched,RCBS_vals.items())))

In [ ]:
upper_bound=np.quantile(list(RCBS_vals.values()),0.75)+5*stats.iqr(list(RCBS_vals.values()))
lower_bound=np.quantile(list(RCBS_vals.values()),0.25)-5*stats.iqr(list(RCBS_vals.values()))
corr_1=list(filter(lambda x: lower_bound<=x<=upper_bound,comp.values()))

In [ ]:
comparison=[stats.mannwhitneyu(list(RCBS_vals.values()),enriched)[1]]
ps=list(map(lambda x: "ns"if x>=0.05 else "$"+f"{x:.2e}"f"{x}"[:f"{x:.2e}".index("-")].replace("e","\cdot10^{%s}"%f"{x:.2e}"[f"{x:.2e}".index("-"):])+"$",comparison))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

sns.set_style("white")

sns.histplot(data=[list(RCBS_vals.values()),enriched],bins=50,kde=True,palette=["#87CEEB","#FFA500"])

ax.axvline(np.median(list(RCBS_vals.values())),linestyle='-',color='k', alpha=0.5)
ax.axvline(np.median(enriched),linestyle='--',color='k', alpha=0.5) 

ax.legend(labels=["Transcriptome", "PB-in"][::-1],ncols=1) 
ax.set_title('Mock')
ax.set_xlabel("RCBS")

ax_sub=fig.add_axes([0.18, 0.4,0.25,0.3])
sns.set_style("whitegrid")

#second=sns.stripplot(data=[corr_1,enriched_cor],color="k", 
            #size=1,alpha=0.5)

second=sns.boxplot(data=[list(RCBS_vals.values()),enriched],showfliers=False,palette=["#87CEEB","#FFA500"])

from statannot import add_stat_annotation
add_stat_annotation(ax=second,x=["Transcriptome", "PB-in"],
                                           y=[np.median(list(RCBS_vals.values())),np.median(enriched)],
                                           box_pairs=[("Transcriptome", "PB-in")],
                                           pvalues=comparison,
                                           text_annot_custom=ps,
                                           text_format="simple",
                                           loc="inside",
                                           perform_stat_test=False,
                                           line_offset_to_box=0.5)

ax_sub.set_xticklabels(["Transcriptome", "PB-in"])

#plt.savefig("name.tiff",dpi=600)